In [28]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoConfig
from datasets import load_dataset
import re

# define 함수 제작(토큰 개수, 특수 토큰 확인)

In [63]:
with open('kor_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().split('\n') # kor_stopwords를 가져옴

In [69]:
# token 개수를 세고, 해당 모델의 특수토큰과 특수토큰의 고유 id, 개수를 출력

def token_count(model, data) :
  tokenizer_model = (model)
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, use_fast = True)
  cleaned_data = [re.sub('[^가-힣\s]+', '', d) for d in data] # 정규식을 이용하여 한글과 공백만 남기고 나머지 문자 제거
  encoded_data = tokenizer(data, max_length=1024, truncation= True)
  result = np.concatenate(encoded_data['input_ids']).tolist()
  
  special_token_ids = []

  for i in tokenizer.special_tokens_map.keys():
      special_token_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.special_tokens_map[i]))
      
  my_dict = dict(zip(tokenizer.special_tokens_map.keys(),special_token_ids))
  
  print(f"{model} 토큰 개수 :",len(result))
  
  key_value = [[key, result.count(value)] for key, value in my_dict.items()]
  df_special = pd.DataFrame(list(my_dict.values()), index=my_dict.keys(), columns=['lambda'])
  df_key_value = pd.DataFrame(key_value, columns = ['Token', 'Count']).set_index('Token')
  
  return pd.concat([df_special, df_key_value], axis = 1)

# DATA SET - english
- cnn_dailymail의 데이터 중 'article' 5개 선정

In [5]:
dataset = load_dataset("cnn_dailymail",version="3.0.0")

Found cached dataset cnn_dailymail (C:/Users/WIN/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [7]:
data = dataset['train'][:5]
text = data['article']

In [8]:
data

{'article': ['LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

In [4]:
# 빠른 로드를 위해 리스트로 저장

text = ['LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he\'ll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I\'ll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe\'s earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say \'kid star goes off the rails,\'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films.  Watch I-Reporter give her review of Potter\'s latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer\'s "Equus." Meanwhile, he is braced for even closer media scrutiny now that he\'s legally an adult: "I just think I\'m going to be more sort of fair game," he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.',
 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman. So, they end up on the ninth floor severely mentally disturbed, but not getting any real help because they\'re in jail. We toured the jail with Leifman. He is well known in Miami as an advocate for justice and the mentally ill. Even though we were not exactly welcomed with open arms by the guards, we were given permission to shoot videotape and tour the floor.  Go inside the \'forgotten floor\' » . At first, it\'s hard to determine where the people are. The prisoners are wearing sleeveless robes. Imagine cutting holes for arms and feet in a heavy wool sleeping bag -- that\'s kind of what they look like. They\'re designed to keep the mentally ill patients from injuring themselves. That\'s also why they have no shoes, laces or mattresses. Leifman says about one-third of all people in Miami-Dade county jails are mentally ill. So, he says, the sheer volume is overwhelming the system, and the result is what we see on the ninth floor. Of course, it is a jail, so it\'s not supposed to be warm and comforting, but the lights glare, the cells are tiny and it\'s loud. We see two, sometimes three men -- sometimes in the robes, sometimes naked, lying or sitting in their cells. "I am the son of the president. You need to get me out of here!" one man shouts at me. He is absolutely serious, convinced that help is on the way -- if only he could reach the White House. Leifman tells me that these prisoner-patients will often circulate through the system, occasionally stabilizing in a mental hospital, only to return to jail to face their charges. It\'s brutally unjust, in his mind, and he has become a strong advocate for changing things in Miami. Over a meal later, we talk about how things got this way for mental patients. Leifman says 200 years ago people were considered "lunatics" and they were locked up in jails even if they had no charges against them. They were just considered unfit to be in society. Over the years, he says, there was some public outcry, and the mentally ill were moved out of jails and into hospitals. But Leifman says many of these mental hospitals were so horrible they were shut down. Where did the patients go? Nowhere. The streets. They became, in many cases, the homeless, he says. They never got treatment. Leifman says in 1955 there were more than half a million people in state mental hospitals, and today that number has been reduced 90 percent, and 40,000 to 50,000 people are in mental hospitals. The judge says he\'s working to change this. Starting in 2008, many inmates who would otherwise have been brought to the "forgotten floor"  will instead be sent to a new mental health facility -- the first step on a journey toward long-term treatment, not just punishment. Leifman says it\'s not the complete answer, but it\'s a start. Leifman says the best part is that it\'s a win-win solution. The patients win, the families are relieved, and the state saves money by simply not cycling these prisoners through again and again. And, for Leifman, justice is served. E-mail to a friend .',
 'MINNEAPOLIS, Minnesota (CNN) -- Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. "The whole bridge from one side of the Mississippi to the other just completely gave way, fell all the way down," survivor Gary Babineau told CNN. "I probably had a 30-, 35-foot free fall. And there\'s cars in the water, there\'s cars on fire. The whole bridge is down." He said his back was injured but he determined he could move around. "I realized there was a school bus right next to me, and me and a couple of other guys went over and started lifting the kids off the bridge. They were yelling, screaming, bleeding. I think there were some broken bones."  Watch a driver describe his narrow escape » . At home when he heard about the disaster, Dr. John Hink, an emergency room physician, jumped into his car and rushed to the scene in 15 minutes. He arrived at the south side of the bridge, stood on the riverbank and saw dozens of people lying dazed on an expansive deck. They were in the middle of the Mississippi River, which was churning fast, and he had no way of getting to them. He went to the north side, where there was easier access to people. Ambulances were also having a hard time driving down to the river to get closer to the scene. Working feverishly, volunteers, EMTs and other officials managed to get 55 people into ambulances in less than two hours. Occasionally, a pickup truck with a medic inside would drive to get an injured person and bring him back up even ground, Hink told CNN. The rescue effort was controlled and organized, he said; the opposite of the lightning-quick collapse. "I could see the whole bridge as it was going down, as it was falling," Babineau said. "It just gave a rumble real quick, and it all just gave way, and it just fell completely, all the way to the ground. And there was dust everywhere and it was just like everyone has been saying: It was just like out of the movies." Babineau said the rear of his pickup truck was dangling over the edge of a broken-off section of the bridge. He said several vehicles slid past him into the water. "I stayed in my car for one or two seconds. I saw a couple cars fall," he said. "So I stayed in my car until the cars quit falling for a second, then I got out real quick, ran in front of my truck -- because behind my truck was just a hole -- and I helped a woman off of the bridge with me. "I just wanted off the bridge, and then I ran over to the school bus. I started grabbing kids and handing them down. It was just complete chaos." He said most of the children were crying or screaming. He and other rescuers set them on the ground and told them to run to the river bank, but a few needed to be carried because of their injuries.  See rescuers clamber over rubble » . Babineau said he had no rescue training. "I just knew what I had to do at the moment." Melissa Hughes, 32, of Minneapolis, told The Associated Press that she was driving home when the western edge of the bridge collapsed under her. "You know that free-fall feeling? I felt that twice," Hughes said. A pickup landed on top of her car, but she was not hurt. "I had no idea there was a vehicle on my car," she told AP. "It\'s really very surreal." Babineau told the Minneapolis Star-Tribune: "On the way down, I thought I was dead. I literally thought I was dead. "My truck was completely face down, pointed toward the ground, and my truck got ripped in half. It was folded in half, and I can\'t believe I\'m alive."  See and hear eyewitness accounts » . Bernie Toivonen told CNN\'s "American Morning" that his vehicle was on a part of the bridge that ended up tilted at a 45-degree angle. "I knew the deck was going down, there was no question about it, and I thought I was going to die," he said. After the bridge settled and his car remained upright, "I just put in park, turned the key off and said, \'Oh, I\'m alive,\' " he said. E-mail to a friend .',
 'WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush\'s colon on Saturday, and "none appeared worrisome," a White House spokesman said. The polyps were removed and sent to the National Naval Medical Center in Bethesda, Maryland, for routine microscopic examination, spokesman Scott Stanzel said. Results are expected in two to three days. All were small, less than a centimeter [half an inch] in diameter, he said. Bush is in good humor, Stanzel said, and will resume his activities at Camp David. During the procedure Vice President Dick Cheney assumed presidential power. Bush reclaimed presidential power at 9:21 a.m. after about two hours. Doctors used "monitored anesthesia care," Stanzel said, so the president was asleep, but not as deeply unconscious as with a true general anesthetic. He spoke to first lady Laura Bush -- who is in Midland, Texas, celebrating her mother\'s birthday -- before and after the procedure, Stanzel said. Afterward, the president played with his Scottish terriers, Barney and Miss Beazley, Stanzel said. He planned to have lunch at Camp David and have briefings with National Security Adviser Stephen Hadley and White House Chief of Staff Josh Bolten, and planned to take a bicycle ride Saturday afternoon. Cheney, meanwhile, spent the morning at his home on Maryland\'s eastern shore, reading and playing with his dogs, Stanzel said. Nothing occurred that required him to take official action as president before Bush reclaimed presidential power. The procedure was supervised by Dr. Richard Tubb, Bush\'s physician, and conducted by a multidisciplinary team from the National Naval Medical Center in Bethesda, Maryland, the White House said. Bush\'s last colonoscopy was in June 2002, and no abnormalities were found, White House spokesman Tony Snow said. The president\'s doctor had recommended a repeat procedure in about five years. A colonoscopy is the most sensitive test for colon cancer, rectal cancer and polyps, small clumps of cells that can become cancerous, according to the Mayo Clinic. Small polyps may be removed during the procedure. Snow said on Friday that Bush had polyps removed during colonoscopies before becoming president. Snow himself is undergoing chemotherapy for cancer that began in his colon and spread to his liver.  Watch Snow talk about Bush\'s procedure and his own colon cancer » . "The president wants to encourage everybody to use surveillance," Snow said. The American Cancer Society recommends that people without high risk factors or symptoms begin getting screened for signs of colorectal cancer at age 50. E-mail to a friend .',
 '(CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. "Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fans have all been hurt by your actions," NFL Commissioner Roger Goodell said in a letter to Vick. Goodell said he would review the status of the suspension after the legal proceedings are over. In papers filed Friday with a federal court in Virginia, Vick also admitted that he and two co-conspirators killed dogs that did not fight well. Falcons owner Arthur Blank said Vick\'s admissions describe actions that are "incomprehensible and unacceptable." The suspension makes "a strong statement that conduct which tarnishes the good reputation of the NFL will not be tolerated," he said in a statement.  Watch what led to Vick\'s suspension » . Goodell said the Falcons could "assert any claims or remedies" to recover $22 million of Vick\'s signing bonus from the 10-year, $130 million contract he signed in 2004, according to The Associated Press. Vick said he would plead guilty to one count of "Conspiracy to Travel in Interstate Commerce in Aid of Unlawful Activities and to Sponsor a Dog in an Animal Fighting Venture" in a plea agreement filed at U.S. District Court in Richmond, Virginia. The charge is punishable by up to five years in prison, a $250,000 fine, "full restitution, a special assessment and 3 years of supervised release," the plea deal said. Federal prosecutors agreed to ask for the low end of the sentencing guidelines. "The defendant will plead guilty because the defendant is in fact guilty of the charged offense," the plea agreement said. In an additional summary of facts, signed by Vick and filed with the agreement, Vick admitted buying pit bulls and the property used for training and fighting the dogs, but the statement said he did not bet on the fights or receive any of the money won. "Most of the \'Bad Newz Kennels\' operations and gambling monies were provided by Vick," the official summary of facts said. Gambling wins were generally split among co-conspirators Tony Taylor, Quanis Phillips and sometimes Purnell Peace, it continued. "Vick did not gamble by placing side bets on any of the fights. Vick did not receive any of the proceeds from the purses that were won by \'Bad Newz Kennels.\' " Vick also agreed that "collective efforts" by him and two others caused the deaths of at least six dogs. Around April, Vick, Peace and Phillips tested some dogs in fighting sessions at Vick\'s property in Virginia, the statement said. "Peace, Phillips and Vick agreed to the killing of approximately 6-8 dogs that did not perform well in \'testing\' sessions at 1915 Moonlight Road and all of those dogs were killed by various methods, including hanging and drowning. "Vick agrees and stipulates that these dogs all died as a result of the collective efforts of Peace, Phillips and Vick," the summary said. Peace, 35, of Virginia Beach, Virginia; Phillips, 28, of Atlanta, Georgia; and Taylor, 34, of Hampton, Virginia, already have accepted agreements to plead guilty in exchange for reduced sentences. Vick, 27, is scheduled to appear Monday in court, where he is expected to plead guilty before a judge.  See a timeline of the case against Vick » . The judge in the case will have the final say over the plea agreement. The federal case against Vick focused on the interstate conspiracy, but Vick\'s admission that he was involved in the killing of dogs could lead to local charges, according to CNN legal analyst Jeffrey Toobin. "It sometimes happens -- not often -- that the state will follow a federal prosecution by charging its own crimes for exactly the same behavior," Toobin said Friday. "The risk for Vick is, if he makes admissions in his federal guilty plea, the state of Virginia could say, \'Hey, look, you admitted violating Virginia state law as well. We\'re going to introduce that against you and charge you in our court.\' " In the plea deal, Vick agreed to cooperate with investigators and provide all information he may have on any criminal activity and to testify if necessary. Vick also agreed to turn over any documents he has and to submit to polygraph tests. Vick agreed to "make restitution for the full amount of the costs associated" with the dogs that are being held by the government. "Such costs may include, but are not limited to, all costs associated with the care of the dogs involved in that case, including if necessary, the long-term care and/or the humane euthanasia of some or all of those animals." Prosecutors, with the support of animal rights activists, have asked for permission to euthanize the dogs. But the dogs could serve as important evidence in the cases against Vick and his admitted co-conspirators. Judge Henry E. Hudson issued an order Thursday telling the U.S. Marshals Service to "arrest and seize the defendant property, and use discretion and whatever means appropriate to protect and maintain said defendant property." Both the judge\'s order and Vick\'s filing refer to "approximately" 53 pit bull dogs. After Vick\'s indictment last month, Goodell ordered the quarterback not to report to the Falcons training camp, and the league is reviewing the case. Blank told the NFL Network on Monday he could not speculate on Vick\'s future as a Falcon, at least not until he had seen "a statement of facts" in the case.  E-mail to a friend . CNN\'s Mike Phelan contributed to this report.']

### BERT

In [5]:
token_count('bert-base-uncased', text)

bert-base-uncased 토큰 개수 : 4017


,lambda,Count
unk_token,100,0
sep_token,102,5
pad_token,0,0
cls_token,101,5
mask_token,103,0


#### Roberta

In [6]:
token_count('roberta-base',text)

roberta-base 토큰 개수 : 3927


,lambda,Count
bos_token,0,5
eos_token,2,5
unk_token,3,0
sep_token,2,5
pad_token,1,0
cls_token,0,5
mask_token,50264,0


#### gpt2-xl

In [7]:
token_count('gpt2-xl',text)

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

gpt2-xl 토큰 개수 : 3919


,lambda,Count
bos_token,50256,0
eos_token,50256,0
unk_token,50256,0


# DATA SET - korean

In [3]:
kor_text = ['제1조 ①대한민국은 민주공화국이다.②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
            '제2조 ①대한민국의 국민이 되는 요건은 법률로 정한다. ②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
            '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
            '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일정책을 수립하고 이를 추진한다.',
            '제5조 ①대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.']

# 헌법 1-5조항

In [2]:
from Korpora import Korpora

nsmc = Korpora.load("nsmc", force_download=True) # nsmc data 다운


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:01, 11.3MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 10.5MB/s]                            


In [31]:
nsmc_text = list(nsmc.train.get_all_texts())
nsmc_text[:6]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.']

## Huggingface tokenizer

### kcbert-base

In [70]:
token_count('beomi/kcbert-base', nsmc_text)

beomi/kcbert-base 토큰 개수 : 2941879


,lambda,Count
unk_token,1,12063
sep_token,3,150000
pad_token,0,0
cls_token,2,150000
mask_token,4,0


### kobigbird

In [71]:
token_count('monologg/kobigbird-bert-base', nsmc_text)

monologg/kobigbird-bert-base 토큰 개수 : 3385915


,lambda,Count
bos_token,5,0
eos_token,6,0
unk_token,1,2710
sep_token,3,150000
pad_token,0,0
cls_token,2,150000
mask_token,4,0


### kogpt

In [43]:
token_count('taeminlee/kogpt2', nsmc_text)

taeminlee/kogpt2 토큰 개수 : 2262044


,lambda,Count
bos_token,0,0
eos_token,1,0
unk_token,5,0
pad_token,3,0


In [51]:
tokenizer = AutoTokenizer.from_pretrained('taeminlee/kogpt2')
encoded_data = tokenizer(nsmc_text, max_length=1024, truncation= True)

In [52]:
print('\n'.join(map(str, encoded_data['input_ids'][:5])), '\n')

[47485, 280, 48212, 192, 7175, 5895, 47643, 47491, 7974, 9092]
[48640, 218, 47602, 1432, 1693, 326, 48123, 3047, 47867, 6828, 22389, 24433, 4804, 10564, 47445, 200, 7159]
[16627, 47543, 189, 35255, 2298, 31708, 8269, 178]
[45194, 2532, 47497, 47974, 47437, 192, 48092, 47639, 47665, 4005, 47441, 630, 192, 47703, 47645, 2151]
[4718, 48191, 47800, 30619, 2682, 47801, 11874, 22144, 15009, 2415, 1069, 47812, 41951, 47690, 48172, 109, 25316, 47487, 47484, 1988, 47494, 4972, 713, 17489, 3616, 37135, 2236, 9596, 104, 48722, 47484, 269] 



## 어절 단위 Tokenizer

In [72]:
import re

def split_tokenize(text):
    
    tokenized_sentences = []
    for i in text:
        text = re.sub('[^가-힣\s]+',' ', i)
        # ^는 문자열의 시작을 의미하며, [^가-힣\s]는 한글이 아니거나 공백이 아닌 문자 하나를 의미합니다. 그리고 +는 앞의 문자 클래스가 1회 이상 반복되어 나타나는 패턴을 의미
        tokens = i.split()
        if tokens not in stopwords:
         tokenized_sentences.extend(tokens)
    
    return len(tokenized_sentences)

In [73]:
split_tokenize(nsmc_text)

1137736